In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Admin\\Desktop\\Rohit\\MachineLearning\\ml-mlops-workflow\\research\\Modeling'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators : int
    max_features : str
    max_depth : int
    criterion : str
    min_samples_split : int
    target_column: str

In [5]:
from carbonfootprint.constants import *
from carbonfootprint.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForest
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            max_features = params.max_features,
            max_depth = params.max_depth,
            criterion = params.criterion,
            min_samples_split = params.min_samples_split,
            target_column = schema.name
        )

        return model_trainer_config

In [7]:
import pandas as pd
import os
from carbonfootprint import logger
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
from sklearn.ensemble import RandomForestRegressor

In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        # rf = RandomForestRegressor()
        rf = RandomForestRegressor(n_estimators = self.config.n_estimators, min_samples_split = self.config.min_samples_split, max_features = self.config.max_features, max_depth = self.config.max_depth, criterion = self.config.criterion)
        rf.fit(train_x, train_y)

        # lr = LinearRegression()
        # lr.fit(train_x, train_y)

        ypred = rf.predict(test_x)
        print('predictionssssss', ypred)

        print("training score = ",rf.score(train_x,train_y))
        print("testing score = ",rf.score(test_x,test_y))
        print('modelname', self.config.model_name)

        joblib.dump(rf, os.path.join(self.config.root_dir, self.config.model_name))

In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-06-04 15:14:57,097: INFO: common: yaml file: c:\Users\Admin\Desktop\Rohit\MachineLearning\ml-mlops-workflow\config\config.yaml loaded successfully]
[2024-06-04 15:14:57,101: INFO: common: yaml file: c:\Users\Admin\Desktop\Rohit\MachineLearning\ml-mlops-workflow\params.yaml loaded successfully]
[2024-06-04 15:14:57,107: INFO: common: yaml file: c:\Users\Admin\Desktop\Rohit\MachineLearning\ml-mlops-workflow\schema.yaml loaded successfully]
[2024-06-04 15:14:57,109: INFO: common: created directory at: artifacts]
[2024-06-04 15:14:57,110: INFO: common: created directory at: c:\\Users\\Admin\\Desktop\\Rohit\\MachineLearning\\ml-mlops-workflow\\artifacts\\model_trainer]


ValueError: could not convert string to float: 'M6'